In [1]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName('Logistic-regression').getOrCreate()

In [3]:
df = spark.read.csv('titanic.csv', inferSchema = True, header = True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = df.select(['Survived','Pclass','Sex',
                     'Age', 'SibSp','Parch','Fare','Embarked'])

In [7]:
my_final_data = my_cols.na.drop()

In [8]:
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol = 'SexIndex')
gender_encoder = OneHotEncoder(inputCol = 'SexIndex', outputCol = 'SexVec')

In [9]:
embark_indexer = StringIndexer(inputCol = 'Embarked', outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex', outputCol = 'EmbarkVec')

In [10]:
assembler = VectorAssembler(inputCols = ['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare']
                           ,outputCol = 'features')

In [11]:
log_reg_titanic = LogisticRegression(featuresCol = 'features',
                                    labelCol = 'Survived')

In [12]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer,
                             gender_encoder, embark_encoder,
                             assembler, log_reg_titanic])

In [13]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [14]:
fit_model = pipeline.fit(train_data)

In [15]:
results = fit_model.transform(test_data)

In [16]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol= 'prediction',
                                       labelCol = 'Survived')

In [17]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [21]:
AUC = my_eval.evaluate(results)
AUC

0.7815279958137101